In [7]:
# import libraries
import pandas as pd
from ruqiya.ruqiya import clean_text
import re
import string
import nltk
import numpy as np
import ktrain
from ktrain import text
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
#remove emojies
import demoji

In [8]:
#read the dataset
delivCOMP =pd.read_excel('delivCOMP_Dataset.xlsx')
#Display the first 5 rows of the dataset
delivCOMP.head()

,Tweet,Lable,Company
0,@CareemCare @lolliklulu الحين اذا العميل اتاخر...,Negative,Careem
1,@CareemCare اتواصلت معاكم بالتطبيق اناخذ مني م...,Negative,Careem
2,@gooole99 @CareemCare انا كمان مو شغال مدري اش...,Negative,Careem
3,@CareemCare @Cz10112 @Linakh32 ترسلون هنا على ...,Negative,Careem
4,@Li1Ra @HRDF_Care @CareemCare وش السالفه ليه معلق,Negative,Careem


In [9]:
#Describe the dataset
delivCOMP.describe()

,Tweet,Lable,Company
count,6050,6074,6075
unique,5308,3,9
top,هنقرستيشن يملك أسوء خدمة عملاء,Negative,HungerStation
freq,6,4628,1869


In [10]:
#count number of Lable of dataset
delivCOMP["Lable"].value_counts()

Negative    4628
Neutral      800
Positive     646
Name: Lable, dtype: int64

In [11]:
#count number of company of dataset
delivCOMP["Company"].value_counts()

HungerStation    1869
Careem           1306
Jahez            1261
Mrsool           1120
ToYou             341
Shgardi            89
Nahdi              57
The Chefz          20
Ninja              12
Name: Company, dtype: int64

In [12]:
#show positive data
datasetpos = delivCOMP[delivCOMP.Lable == 'Positive']
datasetpos.shape

(646, 3)

In [13]:
#show negative data
datasetneg = delivCOMP[delivCOMP.Lable == 'Negative']
datasetneg.shape

(4628, 3)

In [14]:
#show neutral data
datasetneu = delivCOMP[delivCOMP.Lable == 'Neutral']
datasetneu.shape

(800, 3)

In [15]:
#sample of positive data
datasetpos = datasetpos.sample(frac=0.67,random_state=200)
datasetpos.shape

(433, 3)

In [16]:
#connect all above 
delivCOMP = pd.concat([datasetpos, datasetneg, datasetneu], axis=0)
delivCOMP.head()

,Tweet,Lable,Company
4326,@ShgardiKSA أفضل تطبيق 🖤💛,Positive,Shgardi
3417,@HungerStation شكراً هنقرستيشن على سرعة الاستج...,Positive,HungerStation
2357,@HungerStation @badgalyvonna تكفون يا هنقرستيش...,Positive,HungerStation
4336,@ShgardiKSA اسرع شي ف العالم سريع مرره,Positive,Shgardi
4515,@fhdzz_ هنقر افضل ع الاقل ترجع فلوسي لو في غلط...,Positive,HungerStation


In [17]:
#counts each lable
delivCOMP["Lable"].value_counts()

Negative    4628
Neutral      800
Positive     433
Name: Lable, dtype: int64

In [18]:
#check if there is any null value in dataset
delivCOMP.isnull().any(axis=0)

Tweet       True
Lable      False
Company    False
dtype: bool

In [19]:
#Pre-Processing 

#clean and Normalize the text
def cleanTxt(text):
    text = re.sub(r'[A-Za-z0-9٠-٩]+', '', text) #Remove english letters,english NO.,Arabic NO. 
    text = demoji.replace(text, "")  #Remove Emojis
    return text
#Remove repeating character,punctuations,diacritics,stop words,Normalize Arabic
#for clean_text function
delivCOMP['Tweet'] = delivCOMP['Tweet'].astype(str).apply(clean_text)
#for cleanTxt function
delivCOMP['Tweet'] = delivCOMP['Tweet'].astype(str).apply(cleanTxt)

#Remove duplication tweets
delivCOMP.drop_duplicates(subset = 'Tweet', inplace = True)

In [20]:
delivCOMP

,Tweet,Lable,Company
4326,افضل تطبيق,Positive,Shgardi
3417,شكرا هنقرستيشن سرعه الاستجابه والتعويض سب هن...,Positive,HungerStation
2357,تكفون هنقرستيشن اسبوعنا كرف اسعدونا اله يسع...,Positive,HungerStation
4336,اسرع شي العالم سريع,Positive,Shgardi
4515,هنقر افضل الاقل ترجع فلوسي غلط بغير الطلب جا...,Positive,HungerStation
...,...,...,...
5952,انتهي العرض باقي,Neutral,Jahez
5953,استخدم العرض,Neutral,Jahez
5994,الخير ليه اطلب يطلع,Neutral,Jahez
6014,استفسار\n مكن اضافه حجب المطاعم التطبيق مطاعم...,Neutral,Jahez


In [21]:
# print sample of dataset 
delivCOMP["Tweet"].sample(20)

4013     وربي مرا قهروني طلبت فطور مرسول ومرا جوعانه ض...
650           فرق  \nوالرحله  التاريخ والتوقيت ونفس الوجه
4755     الطلب تاخر المندوب ضيع ابغي استرجع المبلغ واك...
561      يتم دفع المستحق  يتم التاجيل الرجاء المساعده ...
3711                    الغيت الطلب ورجعت طلبت ومافي يقبل
5725     حسبي اله عليكم ونعم الوكيل  انكم حرامين مشوار...
4452     طلبته رصيدي البنكي وطلبتوا رقم الايبان وزودتك...
3701     اسف تعرضت لعمليه نصب  مرسول  الطلب مطعم وتم س...
1703                      مهزله صاير مندوبكم  يدور الرياض
1721                                         يتم التواصل 
3073     السلام عليكم  اطلب شي ويوصلني شي  طلبي طلب قه...
3729     عفوا الحمدله جاء مندوب وقبل الطلب  كثير الاحي...
4651                    مافيه خصم طلبت الدفع مالقيت الخصم
622         خانه مطاعم ماتظهر بالتطبيق العلم موجوده سابقا
682                            اتوقع الشغله تطفيش الكباتن
1758    تاخر الطلب طالبه ونص  ماوصل واكلمكم الخاص محد يرد
3934     يخوي الزبون دفع قيمه الطلب وصلت الطلب ولاجاتن...
5499     اتوقع

In [22]:
#split the dataset to train - test
msk = np.random.rand(len(delivCOMP)) < 0.8
df_train = delivCOMP[msk]
df_test = delivCOMP[~msk]
#print train shape
df_train.shape

(3981, 3)

In [23]:
#print test shape
df_test.shape

(1003, 3)

In [24]:
#counts each lable for test dataset
df_test.Lable.value_counts()

Negative    786
Neutral     145
Positive     72
Name: Lable, dtype: int64

In [25]:
#counts each lable for test dataset
df_train.Lable.value_counts()

Negative    3119
Neutral      537
Positive     325
Name: Lable, dtype: int64

In [26]:
# set hyperparameters # used to controll the learning process
maxlen = 64 #يعتمد بشكل عام على adress space 
batch_size = 16
lr = 2e-5 #learning rate
epochs = 3 #training the neural network with all the training data for one cycle

In [27]:
#arabert modelPre-trained Transformers for the Arabic text
MODEL_NAME = 'aubmindlab/bert-base-arabertv01'
t = text.Transformer(MODEL_NAME, maxlen=maxlen)

In [28]:
trn = t.preprocess_train(df_train.Tweet.values, df_train.Lable.values)
tst = t.preprocess_test(df_test.Tweet.values, df_test.Lable.values)

preprocessing train...
language: ar
train sequence lengths:
	mean : 12
	95percentile : 27
	99percentile : 35


Is Multi-Label? False
preprocessing test...
language: ar
test sequence lengths:
	mean : 12
	95percentile : 25
	99percentile : 34


In [29]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=tst, batch_size=batch_size)

In [30]:
#training data
history = learner.fit_onecycle(lr, epochs)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
249/249 [==============================] - 1691s 7s/step - loss: 0.5974 - accuracy: 0.7674 - val_loss: 0.4502 - val_accuracy: 0.8395
Epoch 2/3
249/249 [==============================] - 1569s 6s/step - loss: 0.3904 - accuracy: 0.8671 - val_loss: 0.3834 - val_accuracy: 0.8654
Epoch 3/3
249/249 [==============================] - 1483s 6s/step - loss: 0.2349 - accuracy: 0.9246 - val_loss: 0.3899 - val_accuracy: 0.8714


In [31]:
#classification report for test data
learner.validate(val_data=tst)

32/32 [==============================] - 96s 3s/step
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       786
           1       0.69      0.47      0.56       145
           2       0.79      0.76      0.77        72

    accuracy                           0.87      1003
   macro avg       0.79      0.73      0.75      1003
weighted avg       0.86      0.87      0.86      1003



array([[751,  28,   7],
       [ 69,  68,   8],
       [ 14,   3,  55]], dtype=int64)

In [32]:
p = ktrain.get_predictor(learner.model, t)

In [33]:
p.predict("هنقرستيشن افضل تطبيق توصيل")

1/1 [==============================] - 0s 115ms/step


'Positive'

In [34]:
p.predict("مرسول اسوأ تطبيق توصيل")

1/1 [==============================] - 0s 98ms/step


'Negative'

In [35]:
p.predict("جاهز رجعوا فلوسي")

1/1 [==============================] - 0s 96ms/step


'Negative'

In [36]:
p.predict("شكرا لكم")

1/1 [==============================] - 0s 94ms/step


'Positive'

In [37]:
p.predict("هنقرستيشن حرامية")

1/1 [==============================] - 0s 103ms/step


'Negative'

In [38]:
p.predict("ردوا على الخاص")

1/1 [==============================] - 0s 104ms/step


'Neutral'

In [39]:
p.predict("المندوب تأخر ")

1/1 [==============================] - 0s 100ms/step


'Negative'

In [40]:
p.predict("رهيب")

1/1 [==============================] - 0s 93ms/step


'Positive'

In [41]:
p.predict("ممكن رابط تحميل برنامج جاهز ؟")

1/1 [==============================] - 0s 95ms/step


'Neutral'

In [42]:
ar=ktrain.load_predictor

In [43]:
p.save('ar-bert-model-datasetmarge')

In [44]:
from tkinter import *

In [ ]:
#import all methods and classes from the tkinter
import tkinter as tk

#Create a GUI window
root= tk.Tk()
#set the name of tkinter GUI window
root.title('Arabic Sentiment Analyzer')

canvas1 = tk.Canvas(root, width=400, height=300, relief='raised')
canvas1.pack()


label1 = tk.Label(root, text='Arabic Sentiment Analyzer')
label1.config(font=('helvetica', 14))
canvas1.create_window(200, 25, window=label1)
#create a label : Enter a Tweet 
label2 = tk.Label(root, text='Enter a Tweet :')
label2.config(font=('helvetica', 10))
canvas1.create_window(200, 100, window=label2)

entry1 = tk.Entry(root,width=50) 
canvas1.create_window(200, 140, window=entry1)

# function to Predict sentiments
def get_pre():
    # get a whole input content from text box
    p1 = entry1.get()
    
    label3 = tk.Label(root, text='The Tweet is :', font=('helvetica', 10))
    canvas1.create_window(200, 210, window=label3)
    
    label4 = tk.Label(root, text=p.predict(p1), font=('helvetica', 10, 'bold'))
    canvas1.create_window(200, 230, window=label4)
#create a Predict Button , when user press the button, the command or function affiliated to that button is executed    
button1 = tk.Button(text='Predict', command=get_pre, bg='#002179', fg='white', font=('helvetica', 9, 'bold'))
canvas1.create_window(180, 180, window=button1)
#create a Clear Button , when user press the button, the command or function will deleting the content from the entry box
button2=tk.Button(text='Clear',command=lambda:entry1.delete(0,'end'),bg='#002179', fg='white', font=('helvetica', 9, 'bold'))
canvas1.create_window(235, 180, window=button2)

#start the GUI
root.mainloop()